In [1]:
import csv
import numpy as np
norm = np.linalg.norm

In [2]:
def dot(a, b):
  # Returns scalar
  return np.dot(a.T, b)[0, 0]

def angle(a, b):
  # https://stackoverflow.com/a/13849249
  return np.arccos(np.clip(dot(a/norm(a), b/norm(b)), -1.0, 1.0))

def single_variate_regression(x, y):
  # Assumes x and y are column vectors
  # Returns beta
  return dot(x, y) / dot(x, x)

def compute_residual(x, z_basis):
  gammas = [single_variate_regression(z, x) for z in z_basis]
  return x - sum([gamma * z for gamma, z in zip(gammas, z_basis)]), gammas

def prepend_1(x):
  # prepend 1 to every row in x
  return np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)

In [10]:
with open('../datasets/winequality/winequality-white.csv') as f:
  reader = csv.reader(f, delimiter=';')
  header = next(reader)
  body = np.asarray([[float(value) for value in row] for row in reader])
x = body[:, :-1]
y = body[:, -1:]
N = 1000  # x.shape[0] // 2
train = (x[:N], y[:N])
test = (x[N:], y[N:])

# Compute gram-schmidt orthogonalization
x = prepend_1(train[0])
y = train[1]
zs = [np.ones([x.shape[0], 1])]
Gamma = np.zeros([x.shape[1], x.shape[1]])
Gamma[0, 0] = 1
indices = list(range(1, x.shape[1]))
chosen_indices = [0]  # Features ordered by best to worst fit (smallest angle to y means better fit)
k = 1
print('Best angles:')
while indices:
  cache = []
  angles = []
  for i in indices:
    z, gammas = compute_residual(x[:, i:i+1], zs)
    cache.append((z, gammas))
    angles.append(angle(z, y))
  best_i = np.argmin(angles)
  print(angles[best_i] / (2*np.pi))
  z, gammas = cache[best_i]
  zs.append(z)
  Gamma[k, k] = 1
  Gamma[:k, k] = gammas
  k += 1
  chosen_indices.append(indices[best_i])
  del indices[best_i]
print('')

# check basis is orthogonal
for i in range(len(zs)):
  for j in range(i):
    assert round(dot(zs[i], zs[j]), 5) == 0, dot(zs[i], zs[j])


def subset_regression(k):
  assert k >= 1
  Z = np.concatenate(zs[:k], axis=1)
  D = np.diag(np.linalg.norm(Z, axis=0).reshape(-1))

  # QR decomposition
  Q = np.matmul(Z, np.linalg.inv(D))
  R = np.matmul(D, Gamma[:k, :k])

  # Verify that X = QR
  x, y = train
  x = prepend_1(x)
  idxs = chosen_indices[:k]
  assert np.allclose(x[:, idxs], np.matmul(Q, R))

  # Compute MSE
  beta_hat = np.matmul(np.matmul(np.linalg.inv(R), Q.T), y)
  y_hat = np.matmul(x[:, idxs], beta_hat)
  train_mse = np.power(y - y_hat, 2).mean()

  x, y = test
  x = prepend_1(x)
  y_hat = np.matmul(x[:, idxs], beta_hat)
  test_mse = np.power(y - y_hat, 2).mean()
  print('k={:02d}; Train MSE: {:.4f}; Test MSE {:.4f}'.format(k, train_mse, test_mse))

# Forward stepwise subset selection.
for k in range(1, x.shape[1]):
  # Compute regression with different subset sizes.
  # Take best fitting features in each subset.
  subset_regression(k)

Best angles:
0.23954351380532657
0.24633455913007982
0.24693456271472852
0.24719812156781767
0.24835944166704155
0.24928751499917826
0.25015867793972074
0.2514150591064964
0.25151232938427576
0.253664493964651
0.25410499281157284

k=01; Train MSE: 0.8688; Test MSE 0.7627
k=02; Train MSE: 0.7168; Test MSE 0.6363
k=03; Train MSE: 0.6981; Test MSE 0.6403
k=04; Train MSE: 0.6850; Test MSE 0.6344
k=05; Train MSE: 0.6741; Test MSE 0.6397
k=06; Train MSE: 0.6703; Test MSE 0.6518
k=07; Train MSE: 0.6696; Test MSE 0.6510
k=08; Train MSE: 0.6696; Test MSE 0.6523
k=09; Train MSE: 0.6668; Test MSE 0.6497
k=10; Train MSE: 0.6636; Test MSE 0.6425
k=11; Train MSE: 0.6449; Test MSE 0.6403
